### http://stackoverflow.com/questions/41390572/pyspark-how-to-reference-a-dataframe-when-in-an-udf-on-another-dataframe

In [2]:
import os
import sys

spark_path = "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path
#os.environ['PYSPARK_PYTHON'] = 'C:\Users\Wei Zhong\Anaconda3\envs\python2\python' # <---

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

sc = SparkContext("local")
sqlCtx = SQLContext(sc)

In [11]:
# Generate Random Data
import itertools
import random
student_ids = ['student1', 'student2', 'student3']
subjects = ['Math', 'Biology', 'Chemistry', 'Physics']
random.seed(1)
data = []
 
for (student_id, subject) in itertools.product(student_ids, subjects):
    data.append((student_id, subject, random.randint(0, 100)))

from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
            StructField("student_id", StringType(), nullable=False),
            StructField("subject", StringType(), nullable=False),
            StructField("score", IntegerType(), nullable=False)
    ])
 
# Create DataFrame 
rdd = sc.parallelize(data)
scores = sqlCtx.createDataFrame(rdd, schema)

# create another dataframe
last_name = ["Granger", "Weasley", "Potter"]
data2 = []
for i in range(len(student_ids)):
    data2.append((student_ids[i], last_name[i]))

schema = StructType([
            StructField("student_id", StringType(), nullable=False),
            StructField("last_name", StringType(), nullable=False)
    ])

rdd = sc.parallelize(data2)
lastnames = sqlCtx.createDataFrame(rdd, schema)

In [12]:
scores.show()
lastnames.show()

+----------+---------+-----+
|student_id|  subject|score|
+----------+---------+-----+
|  student1|     Math|   13|
|  student1|  Biology|   85|
|  student1|Chemistry|   77|
|  student1|  Physics|   25|
|  student2|     Math|   50|
|  student2|  Biology|   45|
|  student2|Chemistry|   65|
|  student2|  Physics|   79|
|  student3|     Math|    9|
|  student3|  Biology|    2|
|  student3|Chemistry|   84|
|  student3|  Physics|   43|
+----------+---------+-----+

+----------+---------+
|student_id|last_name|
+----------+---------+
|  student1|  Granger|
|  student2|  Weasley|
|  student3|   Potter|
+----------+---------+



In [55]:
from pyspark.sql.functions import udf
def getLastName(sid):
    tmp_df = lastnames.filter(lastnames.student_id == sid)
    return tmp_df.last_name

getLastName_udf = udf(getLastName, StringType())
scores.withColumn("last_name", getLastName_udf("student_id")).show(10)

Py4JJavaError: An error occurred while calling o567.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 24, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6/python/pyspark/sql/functions.py", line 1563, in <lambda>
    func = lambda _, it: map(lambda x: returnType.toInternal(f(*x)), it)
  File "

In [56]:
data2 = {}
for i in range(len(student_ids)):
    data2[student_ids[i]] = last_name[i]

lastnames = sc.broadcast(data2)
print type(lastnames)

from pyspark.sql.functions import udf
def getLastName(sid):
    return lastnames.value[sid]

getLastName_udf = udf(getLastName, StringType())
scores.withColumn("last_name", getLastName_udf("student_id")).show(10)

<class 'pyspark.broadcast.Broadcast'>
+----------+---------+-----+---------+
|student_id|  subject|score|last_name|
+----------+---------+-----+---------+
|  student1|     Math|   13|  Granger|
|  student1|  Biology|   85|  Granger|
|  student1|Chemistry|   77|  Granger|
|  student1|  Physics|   25|  Granger|
|  student2|     Math|   50|  Weasley|
|  student2|  Biology|   45|  Weasley|
|  student2|Chemistry|   65|  Weasley|
|  student2|  Physics|   79|  Weasley|
|  student3|     Math|    9|   Potter|
|  student3|  Biology|    2|   Potter|
+----------+---------+-----+---------+
only showing top 10 rows



In [ ]:
sc.stop()